# The Reflection Pattern

This chapter reveals a key insight: **AI agents are just automated loops**.

We'll implement the same task two ways — showing how the agent automates what you can build manually.

---

## What is Reflection?

The **Reflection pattern** is a loop where the AI:
1. **Generates** an output
2. **Critiques** its own output
3. **Refines** based on the critique
4. **Repeats** until satisfied

```
┌─────────────────────────────────────────────────────────────────────┐
│                        REFLECTION LOOP                              │
│                                                                     │
│    ┌──────────┐      ┌──────────┐      ┌──────────┐                │
│    │ GENERATE │─────▶│ CRITIQUE │─────▶│  REFINE  │                │
│    │          │      │          │      │          │                │
│    │  Draft   │      │  Check   │      │   Fix    │                │
│    │  output  │      │  issues  │      │  issues  │                │
│    └──────────┘      └────┬─────┘      └────┬─────┘                │
│                           │                 │                       │
│                           │ Issues?         │                       │
│                           │                 │                       │
│                      No ──┴── Yes ──────────┘                       │
│                           │                 ▲                       │
│                           ▼                 │                       │
│                    ┌──────────┐             │                       │
│                    │  OUTPUT  │      (repeat if needed)            │
│                    └──────────┘                                     │
└─────────────────────────────────────────────────────────────────────┘
```

This pattern is validated by research — the **Self-Refine paper** (Madaan et al., 2023) shows it improves task performance by ~20% on average.

**Key insight:** This is exactly what AI agents do internally. The agent node just automates this loop for you.

---

## The Challenge: Constrained Product Description

To demonstrate reflection, we need a task that LLMs typically fail on the first try. Writing with **multiple hard constraints** is perfect:

| Constraint | Requirement |
|------------|-------------|
| **Sentences** | Exactly 3 |
| **Words** | 25–30 total |
| **Keyword** | Contains "sound" exactly twice |
| **Forbidden** | Does NOT contain "music" or "audio" |

**Product:** Wireless Bluetooth Headphones

Try asking ChatGPT to do this in one shot — it almost always fails at least one constraint. This is why reflection matters.

---

## Two Approaches, One Pattern

We'll implement the same reflection loop two ways:

| Version | Who controls the loop? | Best for |
|---------|------------------------|----------|
| **V1: Loop with Exit** | You design the loop | When you need full control |
| **V2: AI Agent** | AI decides when to stop | When you want simplicity |

> **Import via URL** (copy and paste in n8n → Import from URL):
> ```
> https://raw.githubusercontent.com/ezponda/ai-agents-course/main/courses/n8n_no_code/book/_static/workflows/05_reflection_pattern.json
> ```
>
> **Download:** {download}`05_reflection_pattern.json <_static/workflows/05_reflection_pattern.json>`

Both versions are in the same workflow. Each has its own trigger — run them separately and compare!

---

## Version 1: Loop with Exit Condition

You design a workflow that **loops until the constraints pass** (or hits a maximum of 5 iterations).

```
┌──────────────────────────────────────────────────────────────────────────────────────┐
│                                    LOOP                                              │
│  ┌───────────────┐   ┌───────────────┐   ┌───────────────┐   ┌───────────────┐      │
│  │ Generate/     │──▶│   Validate    │──▶│ Parse Result  │──▶│   If Pass?    │──┬──▶│ Output
│  │ Refine (LLM)  │   │    (LLM)      │   │    (Set)      │   │               │  │   │
│  └───────────────┘   └───────────────┘   └───────────────┘   └───────────────┘  │   │
│         ▲                                                            │ No       │   │
│         │                                                            ▼          │   │
│         │                                                    ┌───────────────┐  │   │
│         └────────────────────────────────────────────────────│  If < Max?    │──┘   │
│                              (loop back)                     └───────────────┘      │
└──────────────────────────────────────────────────────────────────────────────────────┘
```

**Pros:** Full control, predictable behavior, easy to debug
**Cons:** More complex to build

### Meet the Node: If (Conditional)

The **If** node checks a condition and sends data to one of two outputs.

| Property | Description |
|----------|-------------|
| **Purpose** | Route data based on a condition |
| **Outputs** | True branch (top) and False branch (bottom) |
| **Common use** | Exit loops, handle errors, route by value |

In V1, we use two If nodes:
- **If Pass?** — checks if `all_pass` is true (exit the loop)
- **If < Max?** — checks if `iteration < 5` (prevents infinite loops)

### Parsing the Validation Result

The Validate LLM returns JSON like `{"all_pass": true, ...}`. We need to extract the `all_pass` value for the If node.

Instead of complex JSON parsing, we use a simple **string check** in an expression:

```
{{ $json.text.includes('"all_pass": true') || $json.text.includes('"all_pass":true') }}
```

This returns `true` if the LLM said all constraints passed, `false` otherwise.

**Why this works:**
- The LLM is instructed to output ONLY JSON
- We just need to know if `all_pass` is `true` — no need to parse the full object
- String matching is simpler and more reliable than JSON parsing in expressions

### How the Loop Works

The key to looping in n8n: **connect the output of a node back to an earlier node**.

```
Generate/Refine ◄──────────────────────────────────┐
     │                                             │
     ▼                                             │
  Validate                                         │
     │                                             │
     ▼                                             │
  Parse Result (Set)                               │
     │                                             │
     ▼                                             │
  If Pass? ───Yes──▶ Output                        │
     │                                             │
     No                                            │
     │                                             │
     ▼                                             │
  If < Max? ───Yes──▶ Prepare Refine ──────────────┘
     │
     No (guardrail: max 5 iterations)
     │
     ▼
  Output (max reached)
```

**Important:** The "If < Max?" node is a **guardrail** that prevents infinite loops. Always include this when building loops!

### Node-by-Node Walkthrough (V1)

| Node | Type | What it does |
|------|------|-------------|
| **Run: V1 Loop** | Manual Trigger | Starts Version 1 |
| **Input — Product (V1)** | Set | Creates `product`, `constraints`, `iteration=0`, `draft=""` |
| **Generate/Refine (V1)** | Basic LLM Chain | Generates or refines based on whether `draft` is empty |
| **Store Draft (V1)** | Set | Saves draft and increments iteration |
| **Validate (V1)** | Basic LLM Chain | Checks constraints, returns JSON |
| **Parse Validation (V1)** | Set | Extracts `all_pass` boolean using string check |
| **If Pass?** | If | Routes to Output if `all_pass`, else continues |
| **If < Max?** | If | Guardrail: checks if iteration < 5 |
| **Prepare Refine (V1)** | Set | Formats data for next iteration |
| **Output (V1)** | Set | Returns `final_description`, `iterations`, `approach` |
| **Output Max (V1)** | Set | Returns result when max iterations reached |

---

## Version 2: AI Agent

The simplest to build: let the agent handle the loop.

```
┌──────────────────────────────────────────────────────────────────────────────┐
│                              AI AGENT                                        │
│                                                                              │
│    ┌─────────────────────────────────────────────────────────────────┐      │
│    │                    INTERNAL LOOP                                │      │
│    │                                                                 │      │
│    │   ┌──────────┐      ┌──────────────┐      ┌──────────┐         │      │
│    │   │  THINK   │─────▶│ Call Tool:   │─────▶│  CHECK   │         │      │
│    │   │          │      │  Validator   │      │          │         │      │
│    │   │ "Draft   │      │  (Python)    │      │ "all_pass │──No────┤      │
│    │   │  a desc" │      │              │      │  true?"  │         │      │
│    │   └──────────┘      └──────────────┘      └────┬─────┘         │      │
│    │         ▲                                      │               │      │
│    │         │                                      │ Yes           │      │
│    │         └──────────────────────────────────────┘               │      │
│    │                                                                 │      │
│    └─────────────────────────────────────────────────────────────────┘      │
│                                                                              │
│    ┌─────────────┐                                                          │
│    │  Validator  │ ◄── Python tool that checks constraints                  │
│    │    Tool     │                                                          │
│    └─────────────┘                                                          │
└──────────────────────────────────────────────────────────────────────────────┘
```

**Pros:** Simplest to build, agent adapts strategy
**Cons:** Less predictable, may take more iterations

### The Validator Tool (Python)

Instead of using an LLM to validate, V2 uses a **Python Code tool** — a function the agent can call:

```python
import re

# Get the description from the agent's input
description = _input.first().json.get('query', '') or _input.first().json.get('input', '')

# Count sentences (split by . ! ?)
sentences = [s for s in re.split(r'[.!?]+', description) if s.strip()]
sentence_count = len(sentences)

# Count words
words = [w for w in description.split() if w]
word_count = len(words)

# Count "sound" occurrences (case insensitive)
sound_matches = re.findall(r'\bsound\b', description.lower())
sound_count = len(sound_matches)

# Check for forbidden words
has_forbidden = bool(re.search(r'\b(music|audio)\b', description, re.IGNORECASE))

# Check all constraints
issues = []
if sentence_count != 3:
    issues.append(f'Sentences: {sentence_count} (need exactly 3)')
if word_count < 25 or word_count > 30:
    issues.append(f'Words: {word_count} (need 25-30)')
# ...

return [{'json': {'all_pass': len(issues) == 0, 'issues': issues}}]
```

**Why a Python tool instead of an LLM?**
- **Faster:** No API call needed
- **Cheaper:** No tokens used
- **Accurate:** Code counts exactly right (LLMs often miscount)

### Node-by-Node Walkthrough (V2)

| Node | Type | What it does |
|------|------|-------------|
| **Run: V2 Agent** | Manual Trigger | Starts Version 2 |
| **Input — Product (V2)** | Set | Creates `product` and `constraints` |
| **Reflection Agent (V2)** | AI Agent | Writes, validates, refines in a loop |
| **Validator Tool (Python)** | Python Code Tool | Python function that checks constraints |
| **Output (V2)** | Set | Returns `final_description` and `approach` |

**Sub-nodes connected to the Agent:**
- **OpenRouter (V2)** — the LLM brain
- **Validator Tool (Python)** — the constraint checker

### Prompt Used (V2)

**System Message:**
```
You are a product copywriter who writes precise descriptions.

Process:
1. Write a draft description
2. Use the Validator tool to check ALL constraints
3. If any constraint fails, revise and validate again
4. Repeat until all_pass is true
5. Return the final description

Rules:
- Count words carefully
- Call the Validator tool at most 5 times
- When all_pass is true, return ONLY the final description
```

Notice: the system message describes the reflection loop explicitly. The agent follows the same pattern we built manually in V1!

---

## The Insight: Agents Are Automated Loops

Compare what each version automates:

| Version | You design... | n8n handles... | AI handles... |
|---------|---------------|----------------|---------------|
| **V1: Loop** | The loop structure, exit condition | Looping, iteration count | Text generation |
| **V2: Agent** | The goal and tools | Everything | The entire loop |

**The agent node is not magic.** It's automating the same Generate → Critique → Refine loop you built by hand.

```
V1 Loop:        You ──────────▶ n8n ──────────────▶ LLM
                (design loop)   (runs loop)

V2 Agent:       You ──▶ n8n Agent Node ───────────▶ LLM
                (goal)  (runs loop internally)
```

---

## When to Use Each Approach

| Situation | Best approach |
|-----------|---------------|
| **You need full control over each step** | V1: Loop |
| **You want to debug exactly what's happening** | V1: Loop |
| **Task is ambiguous, you want AI to adapt** | V2: Agent |
| **You want the simplest implementation** | V2: Agent |

**General rule:** Start with V2 (agent). If you need more control or debugging, build V1.

---

## Try It Yourself

1. **Run both versions** on the same product
2. **Compare:** How many iterations did each take? Which produced the best output?
3. **Change the constraints:** Make them harder (e.g., exactly 27 words) and run again
4. **Observe the agent:** In V2, click the agent node to see its internal tool calls

**Challenge:** Modify V1 to allow up to 10 iterations. Does it ever need that many?

---

## Key Takeaways

1. **Reflection = Generate → Critique → Refine → Repeat**
2. **You can build the loop manually** (V1) or **let the agent handle it** (V2)
3. **Agents are not magic** — they automate the same loop you can build by hand
4. **Always include guardrails** (max iterations) to prevent infinite loops
5. **Python tools are useful** when you need precise validation (LLMs miscount)

**Next:** In the **First AI Agent** chapter, we'll explore more agent capabilities — memory, multiple tools, and conversation.